在这一步结束时，您将理解过拟合和欠拟合的概念，并能够应用这些想法使您的模型更精确。

# 模型实验

既然您有了一个可靠的方式来测量模型的准确性，您可以尝试使用其他模型并查看哪个能够提供最佳的预测。但是，您可以为模型提供哪些备选方案呢？

在scikit-learn的[文档](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)中，您可以看到决策树模型有许多选项（您长时间内可能都不需要或不想用到）。最重要的选项决定了树的深度。请回忆一下[本课程第一篇文章](https://www.kaggle.com/dansbecker/how-models-work)中介绍的，决策树的深度是一种度量它在进行预测前多少次分裂的指标。这是一棵相对较浅的树。

![Depth 2 Tree](https://images.weserv.nl/?url=i.imgur.com/R3ywQsR.png)

在实践中，一棵树在顶层（所有房屋）和一个叶子节点之间分裂成10个部分并不罕见。随着树越来越深，数据集被切割成房屋更少的叶子节点。如果一棵树只有一个分叉，它将数据分为两组。如果每个组再次分叉，我们将得到4组房屋。如果我们通过在每一层添加更多的分叉来不断加倍分组数量，那么到达第10个级别时，我们将拥有\\(2^{10}\\)个房屋组。即1024个叶子。

当我们将房屋分配给许多叶子节点时，每个叶子节点中也有较少的房屋。房屋非常少的叶子节点将给出非常接近这些房屋实际值的预测，但它们可能会为新数据提供非常不可靠的预测（因为每个预测仅基于几个房屋）。

这种现象被称为 **过度拟合**，其中模型几乎完美地匹配训练数据，但在验证和其他新数据上表现不佳。相反，如果树非常浅，它就不会将房屋分成非常不同的群组。

在极端情况下，如果一棵树只把房子分成2或4组，那么每个组仍然有多种类型的房屋。得到的预测可能对大多数房屋而言都偏差很大，即使在训练数据中也是如此（由于同样的原因，验证也会很糟糕）。当模型无法捕捉数据中的重要差异和模式时，所以它在训练数据中表现不佳，那就是 **欠拟合**。

由于我们关心新数据的准确性，我们从验证数据中估计它，我们想要找到欠拟合和过度拟合之间的最佳平衡点。从可视化的角度来看，我们想要在下图中（红色）验证曲线的最低点。

![underfitting_overfitting](https://images.weserv.nl/?url=i.imgur.com/AXSEOfI.png)

# 例子

有一些控制树深度的替代方法，许多方法允许通过树的一些路线比其他路线具有更大的深度。但是 *max _leaf_node* 参数提供了一种非常合理的方法来控制过拟合和不适合。我们允许模型生成的叶子越多，我们从上图中的欠拟合区域越容易移动到过拟合区域。

我们可以使用一个实用函数来帮助比较不同值 *max_leaf_node* 的 MAE 得分:


In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

数据被加载到 **train_X** 、**val_X** 、**train_y** 和 **val_y** 中，使用您已经看到的代码(以及您已经编写的代码)。

In [2]:
# 数据加载代码此时运行
import pandas as pd

# Load data
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

我们可以使用 for 循环来比较用不同值构建的模型对 *max_leaf_node* 的准确性。

In [3]:
# 将 MAE 与 max_leaf_node 的不同值进行比较
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  347380
Max leaf nodes: 50  		 Mean Absolute Error:  258171
Max leaf nodes: 500  		 Mean Absolute Error:  243495
Max leaf nodes: 5000  		 Mean Absolute Error:  254983



在所列出的选项中，500张是最佳的叶子数量

---

# 总结

Here's the takeaway: Models can suffer from either:
- **Overfitting:** capturing spurious patterns that won't recur in the future, leading to less accurate predictions, or 
- **Underfitting:** failing to capture relevant patterns, again leading to less accurate predictions. 

We use **validation** data, which isn't used in model training, to measure a candidate model's accuracy. This lets us try many candidate models and keep the best one. 

以下是我们的要点: 模型可能会遭遇以下两种情况:
- **过度拟合:** 捕获到在未来不会重现的假模式，导致预测不太准确，或者
- **欠拟合:** 未能捕捉到相关的模式，再次导致预测不准确。
我们模型训练中没有使用 **验证数据** 来衡量候选模型的准确性。这让我们可以尝试许多候选模型，并保持最好的一个。

# 练习时间

试试 **[优化先前建立的模型](https://www.kaggle.com/kernels/fork/1259126)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-machine-learning/discussion) to chat with other learners.*